In [1]:
import jams
import os
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm

import interpreter as itp

In [121]:
## Change This Cell!
gs_path = "/Users/tom/Music/GuitarSet/"
anno_dir = "annotation/"
excerpt = "01_SS3-98-C_comp.jams"

excerpts = glob.glob(gs_path+anno_dir+'*.jams')
test_jams_path = glob.glob(gs_path+anno_dir+excerpt)[0]

In [134]:
jams.load(test_jams_path)

<JAMS(file_metadata=<FileMetadata(...)>,
      annotations=[17 annotations],
      sandbox=<Sandbox(...)>)>

In [128]:
def add_key_anno(jam):
    '''Add a key annotation according to the filename
    
    Parameters
    ----------
    jam : jams.JAMS
    '''
    # Build key_mode value string
    track_info_list = jam.file_metadata.title.split('_')
    key_info_list = track_info_list[1].split('-')
    tonic = key_info_list[2]
    mode = 'minor' if key_info_list[0][-1] == '2' else 'major'
    key_mode = ':'.join([tonic, mode])
    
    key_ann = jam.search(namespace='key_mode')
    if len(key_ann):
        print('key annotation already exist for file {}. len(key_ann) = {}'.format(
            os.path.basename(jam_path), len(key_ann)))
    else:
        key_anno = jams.Annotation(namespace='key_mode', duration=jam.file_metadata.duration)
        key_anno.append(time=0, duration=jam.file_metadata.duration, value=key_mode, confidence=1)
        jam.annotations.append(key_anno)

In [129]:
str_midi_dict = {0: 40, 1: 45, 2: 50, 3: 55, 4: 59, 5: 64}

def clean_bad_fret(jam):
    '''Clean the jams annotations to get rid of notes having a negative fret number.
    
    Parameters
    ----------
    jam : jams.JAMS
    
    Returns
    -------
    log : list [source, i, value, fret]
        a log of additional information
    '''
    log = []
    for source in range(6):
        anno_pair = jam.search(data_source=str(source))
        note_ann = anno_pair.search(namespace='note_midi')[0]
        pt_ann = anno_pair.search(namespace='pitch_contour')[0]
        
        per_string_output = inspect_anno_for_bad_fret(note_ann)
        bad_note_indices = [i for _, i, _, _ in per_string_output]
        clean_anno_pair(note_ann, pt_ann, bad_note_indices)
        
        log.extend(per_string_output)
    return log
    
def inspect_anno_for_bad_fret(note_anno, tolerance=0.5):
    output = []
    source = int(note_anno.annotation_metadata.data_source)
    for i, obs in enumerate(note_anno):
        if obs.value - str_midi_dict[source] + tolerance < 0:
            output.append([source, i, obs.value, obs.value - str_midi_dict[source]])
        if obs.value - str_midi_dict[source] > 20:
            output.append([source, i, obs.value, obs.value - str_midi_dict[source]])
    return output

def clean_anno_pair(note_ann, pt_ann, bad_note_indices):
    index_ann = jams.Annotation(namespace='segment_open', duration=note_ann.duration)
    note_data = note_ann.pop_data()
    pt_data = pt_ann.pop_data()
    new_idx = 0
    
    for idx, note in enumerate(note_data):
        if idx not in bad_note_indices:
            note_ann.data.add(note)
            index_ann.append(time=note.time, duration=note.duration, value=new_idx)
            new_idx += 1
            
    for obs in pt_data:
        note_idx_list = index_ann.to_samples([obs.time])[0]
        if len(note_idx_list) > 0:
            new_value = obs.value
            new_value['index'] = int(note_idx_list[0])
            pt_ann.append(time=obs.time, duration=obs.duration,
                          value=new_value)

In [133]:
for jam_path in tqdm(excerpts):
    jam = jams.load(jam_path)
    add_key_anno(jam)
    clean_bad_fret(jam)
    jam.save(jam_path)
#     if output:
#         print('--------------------')
#         print(os.path.basename(jam_path))
# #         for source, i, value, fret in output:
# #             print('{} string, note {}, midi {}, fret {}'.format(source, i, value, fret))
# b        print('--------------------\n')



  0%|          | 0/360 [00:00<?, ?it/s]

  0%|          | 1/360 [00:08<49:20,  8.25s/it]

  1%|          | 2/360 [00:09<36:51,  6.18s/it]

  1%|          | 3/360 [00:15<36:22,  6.11s/it]

  1%|          | 4/360 [00:16<27:06,  4.57s/it]

  1%|▏         | 5/360 [00:32<46:25,  7.85s/it]

  2%|▏         | 6/360 [00:35<39:13,  6.65s/it]

  2%|▏         | 7/360 [00:39<34:02,  5.79s/it]

  2%|▏         | 8/360 [00:42<29:04,  4.96s/it]

  2%|▎         | 9/360 [00:46<26:15,  4.49s/it]

  3%|▎         | 10/360 [00:47<21:09,  3.63s/it]

  3%|▎         | 11/360 [00:48<16:29,  2.84s/it]

  3%|▎         | 12/360 [00:50<14:32,  2.51s/it]

  4%|▎         | 13/360 [00:57<22:28,  3.88s/it]

  4%|▍         | 14/360 [00:58<17:34,  3.05s/it]

  4%|▍         | 15/360 [00:59<14:17,  2.48s/it]

  4%|▍         | 16/360 [01:03<17:03,  2.98s/it]

  5%|▍         | 17/360 [01:05<13:58,  2.44s/it]

  5%|▌         | 18/360 [01:12<22:26,  3.94s/it]

  5%|▌         | 19/360 [01:13<17:59,  3.17s/it]

  6%|▌         |

 40%|████      | 145/360 [08:49<13:56,  3.89s/it]

 41%|████      | 146/360 [08:57<18:11,  5.10s/it]

 41%|████      | 147/360 [09:00<16:11,  4.56s/it]

 41%|████      | 148/360 [09:02<13:16,  3.75s/it]

 41%|████▏     | 149/360 [09:11<18:49,  5.35s/it]

 42%|████▏     | 150/360 [09:12<13:51,  3.96s/it]

 42%|████▏     | 151/360 [09:13<11:24,  3.28s/it]

 42%|████▏     | 152/360 [09:16<10:16,  2.96s/it]

 42%|████▎     | 153/360 [09:17<08:10,  2.37s/it]

 43%|████▎     | 154/360 [09:21<09:44,  2.84s/it]

 43%|████▎     | 155/360 [09:23<09:43,  2.84s/it]

 43%|████▎     | 156/360 [09:25<08:27,  2.49s/it]

 44%|████▎     | 157/360 [09:29<09:22,  2.77s/it]

 44%|████▍     | 158/360 [09:31<08:53,  2.64s/it]

 44%|████▍     | 159/360 [09:33<08:17,  2.48s/it]

 44%|████▍     | 160/360 [09:34<06:54,  2.07s/it]

 45%|████▍     | 161/360 [09:37<08:06,  2.44s/it]

 45%|████▌     | 162/360 [09:39<07:35,  2.30s/it]

 45%|████▌     | 163/360 [09:43<09:01,  2.75s/it]

 46%|████▌     | 164/360 [09:49

 80%|████████  | 288/360 [17:12<06:13,  5.19s/it]

 80%|████████  | 289/360 [17:17<05:56,  5.03s/it]

 81%|████████  | 290/360 [17:18<04:22,  3.75s/it]

 81%|████████  | 291/360 [17:23<05:00,  4.36s/it]

 81%|████████  | 292/360 [17:24<03:43,  3.29s/it]

 81%|████████▏ | 293/360 [17:26<03:19,  2.98s/it]

 82%|████████▏ | 294/360 [17:36<05:24,  4.92s/it]

 82%|████████▏ | 295/360 [17:39<04:46,  4.40s/it]

 82%|████████▏ | 296/360 [17:45<05:14,  4.91s/it]

 82%|████████▎ | 297/360 [17:55<06:38,  6.33s/it]

 83%|████████▎ | 298/360 [17:58<05:26,  5.27s/it]

 83%|████████▎ | 299/360 [18:04<05:48,  5.71s/it]

 83%|████████▎ | 300/360 [18:08<05:06,  5.11s/it]

 84%|████████▎ | 301/360 [18:15<05:39,  5.75s/it]

 84%|████████▍ | 302/360 [18:24<06:19,  6.54s/it]

 84%|████████▍ | 303/360 [18:28<05:37,  5.93s/it]

 84%|████████▍ | 304/360 [18:29<04:11,  4.49s/it]

 85%|████████▍ | 305/360 [18:31<03:22,  3.69s/it]

 85%|████████▌ | 306/360 [18:32<02:33,  2.85s/it]

 85%|████████▌ | 307/360 [18:44